# Reporte de espacio IPv4 no anunciado

Este reporte incluye información acerca de espacio IPv4 asignad por LACNIC pero no visible en la tabla de enrutamiento global.

## Metodología

Se toman las siguientes fuentes de datos:

- "delegated-extended" de LACNIC
- RIPE RIS IPv4 Dump

El algoritmo de alto nivel es:

1. Recorrer el delegated extended
   1. Para cada organización, tomar todas sus asignaciones
   2. Producir un agregado de las mismas
2. Recorrer el producto de (1)
   1. Para cada ruta de RIS, identificar el agregado de asignaciones de donde proviene, y de allí la organización a la que pertenece
3. Recorrer el producto de (2)
   1. Para cada agregado, agregar todas las rutas que le corresponden y de allí calcular cuanto del total todas esas rutas cubren.
   
## Producto del algoritmo

El producto es un archivo separado por pipes con la siguiente estructura:

```
orgid|prefix|visible|dark|total
20118|24.232.0.0/16|65536|0|65536
272078|45.4.0.0/22|1024|0|1024
271038|45.4.4.0/22|1024|0|1024
261330|45.4.8.0/22|1024|0|1024

...
...
258668|45.4.60.0/22|1024|0|1024
258825|45.4.64.0/22|1024|0|1024
```

Las columnas:

- orgid: coincide con el opaque-id del delegated-extended, identifica la organización
- prefijo: prefijo IPv4 
- visible: cantidad de direcciones de este prefijo visibles (es decir, contenidas en rutas)
- dark: cantidad de direcciones no-visibles
- total: cantidad total de direcciones contenidas en el prefijo



## Resultados

### R1: Cantidad de asignaciones completamente invisibles

En este resultado se listan las asignaciones completamente invisibles (es decir, sin ninguna ruta que las cubra) y las organizaciones que las tienen.

In [110]:
import pandas
import sqlite3

w_date = '20191110'
conn = sqlite3.connect(":memory:")

inf = "s1_invisible_prefixes-{}.csv".format(w_date)
df = pandas.read_csv(inf, delimiter='|')
orgs = pandas.read_csv("orgs_lacnic.csv")
df.to_sql("invr", conn, if_exists='append', index=False)

dark_t = df['dark'].sum()
visible_t = df['visible'].sum()
total_t = df['total'].sum()

df.head()

,orgid,prefix,visible,dark,total
0,20118,24.232.0.0/16,65536,0,65536
1,272078,45.4.0.0/22,1024,0,1024
2,271038,45.4.4.0/22,1024,0,1024
3,261330,45.4.8.0/22,1024,0,1024
4,265673,45.4.12.0/22,1024,0,1024


In [111]:
# orgs.query("orgid == 65334")

### R2. Cantidad de asignaciones invisibles 

In [112]:
nva = df.query("visible == 0 and total > 0").sort_values("total", ascending=False)

print("Cantidad de asignaciones no visibles: {}".format(len(nva['total'])))
print("Cantidad de direcciones no visibles en asignaciones completamente invisibles: {}".format(nva['dark'].sum()) )

Cantidad de asignaciones no visibles: 562
Cantidad de direcciones no visibles en asignaciones completamente invisibles: 2504960


### R2.1 Cantidad de asignaciones invisibles mayores o iguales a un /16

In [113]:
nva_21 = df.query("visible == 0 and total > 16384").sort_values("total", ascending=False)

print("Cantidad de asignaciones grandes no visibles: {}".format(len(nva_21['total'])))
print("Cantidad de direcciones no visibles en asignaciones GRANDES completamente invisibles: {}".format(nva_21['dark'].sum()) )

Cantidad de asignaciones grandes no visibles: 26
Cantidad de direcciones no visibles en asignaciones GRANDES completamente invisibles: 1900544


In [114]:
print("Valor aproximado de mercado de las direcciones no visibles en bloques grandes: {} usd".format(nva['total'].sum(axis=0)*15))

Valor aproximado de mercado de las direcciones no visibles en bloques grandes: 37574400 usd


### R2.1: Listado de asignaciones grandes no visibles

In [115]:
nva.to_csv("s1_bigallocs_notvisible-{}.csv".format(w_date, delimiter='|'))

# orgs['orgid'] = pandas.to_numeric(orgs['orgid'])

nva_b = nva.merge(orgs, on='orgid', how='left')

nva_b[['prefix', 'dark','visible', 'handle','name']].head(26)

,prefix,dark,visible,handle,name
0,186.172.0.0/14,262144,0,CL-CCSA39-LACNIC,CLARO CHILE S.A.
1,186.164.0.0/15,131072,0,VE-TCCA-LACNIC,"TELEFONICA VENEZOLANA, C.A."
2,186.98.0.0/15,131072,0,CO-CTSE-LACNIC,COLOMBIA TELECOMUNICACIONES S.A. ESP
3,186.157.0.0/16,65536,0,AR-CCTI1-LACNIC,AMX Argentina S.A.
4,148.242.0.0/16,65536,0,MX-BNMS-LACNIC,"Banco Nacional de Mexico, S.A."
5,140.191.0.0/16,65536,0,AR-UNDP1-LACNIC,United Nations Development Programme
6,167.134.0.0/16,65536,0,VE-PESA-LACNIC,Pequiven S.A.
7,167.28.0.0/16,65536,0,CL-GLSA2-LACNIC,BANCO DEL ESTADO DE CHILE
8,166.110.0.0/16,65536,0,CL-BAED-LACNIC,Banco de A. Edwards
9,164.98.0.0/16,65536,0,CL-EJCH-LACNIC,Ejercito de Chile


In [116]:
print("Cantidad de IPs oscuras contenida en las asignaciones >/16 completamente invisibles: {}".format(nva_b['dark'].sum()))
print("% respecto al total de IPs oscuras: {0:.2f}%".format(nva_b['dark'].sum()/dark_t*100))

Cantidad de IPs oscuras contenida en las asignaciones >/16 completamente invisibles: 2504960
% respecto al total de IPs oscuras: 18.75%


### R3. Porcentaje de espacio no visible respecto del espacio asignado

In [117]:

print("El porcentaje de espacio asignado pero no visible: {0:.2f}%".format(dark_t / total_t * 100))
print("La cantidad de IPs asignadas es: {}".format(total_t))
print("La cantidad de IPs visibles  es: {}".format(visible_t))
print("La cantidad de IPs oscuras   es: {}".format(dark_t))

El porcentaje de espacio asignado pero no visible: 8.06%
La cantidad de IPs asignadas es: 165655552
La cantidad de IPs visibles  es: 152299274
La cantidad de IPs oscuras   es: 13356278


### R4. Cantidad de asignaciones con mayor espacio oscuro, pero parcialmente visibles

In [118]:
nva_parcial = df.query("dark>0 and visible>0").sort_values('dark', ascending=False)
nvb_parcial = nva_parcial.merge(orgs, on='orgid', how='left')

In [119]:
nva_parcial_agg = nva_parcial.groupby('orgid')
print("Direcciones ocultas contenidas en asignaciones parcialmente visibles: {}".format(nva_parcial['dark'].sum()))
nvb_parcial.sort_values('dark', ascending=False).head(20)

Direcciones ocultas contenidas en asignaciones parcialmente visibles: 10851318


,orgid,prefix,visible,dark,total,name,handle,country,city
0,21461,201.96.0.0/11,1464064,633088,2097152,Uninet S.A. de C.V.,MX-USCV4-LACNIC,MX,Tlalpan
1,21461,189.128.0.0/10,3690496,503808,4194304,Uninet S.A. de C.V.,MX-USCV4-LACNIC,MX,Tlalpan
2,21461,187.192.0.0/11,1617408,479744,2097152,Uninet S.A. de C.V.,MX-USCV4-LACNIC,MX,Tlalpan
3,21461,187.128.0.0/11,1703936,393216,2097152,Uninet S.A. de C.V.,MX-USCV4-LACNIC,MX,Tlalpan
4,35073,191.104.0.0/13,131072,393216,524288,COLOMBIA TELECOMUNICACIONES S.A. ESP,CO-CTSE-LACNIC,CO,BOGOTA
5,21461,187.224.0.0/12,673792,374784,1048576,Uninet S.A. de C.V.,MX-USCV4-LACNIC,MX,Tlalpan
6,20118,181.96.0.0/12,716800,331776,1048576,Telecom Argentina S.A.,AR-TAST-LACNIC,AR,Buenos Aires
7,21461,189.224.0.0/11,1819136,278016,2097152,Uninet S.A. de C.V.,MX-USCV4-LACNIC,MX,Tlalpan
8,236102,181.216.0.0/13,262144,262144,524288,CLARO S.A.,BR-CLSA7-LACNIC,BR,São Paulo
9,20118,181.0.0.0/12,808448,240128,1048576,Telecom Argentina S.A.,AR-TAST-LACNIC,AR,Buenos Aires


In [120]:
nvb_parcial.groupby('name').sum().sort_values('dark', ascending=False).head(10)
# nvb_parcial.groupby('name').head(10)

,orgid,visible,dark,total
name,,,,
Uninet S.A. de C.V.,407759,12292864,3052800,15345664
"TELEFONICA VENEZOLANA, C.A.",230850,546816,1050624,1597440
COLOMBIA TELECOMUNICACIONES S.A. ESP,491022,1666048,955392,2621440
Telecom Argentina S.A.,281652,4053504,943616,4997120
CLARO S.A.,5430346,5158656,706816,5865472
"Axtel, S.A.B. de C.V.",514920,1162752,426496,1589248
CORPORACION NACIONAL DE TELECOMUNICACIONES - CNT EP,272100,285184,413184,698368
Cablemas Telecomunicaciones SA de CV,637896,643328,298752,942080
"Mega Cable, S.A. de C.V.",218820,2106112,257280,2363392


In [121]:
nva_parcial_agg.get_group(35073).sum()

orgid                                                 491022
prefix     191.104.0.0/13186.168.0.0/14152.200.0.0/14152....
visible                                              1666048
dark                                                  955392
total                                                2621440
dtype: object

In [122]:
nvb_parcial.groupby('name').sum().sum()

orgid      74390136
visible    39484426
dark       10742262
total      50226688
dtype: int64